[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/karnsaurabhkumar/soscipy/blob/master/examples/1_Getting%20Started.ipynb)


# SoSciPy
Soscipy is a python library to simplify working with data specially in social sciences. While there are several packages out there, I have personally found it difficult to find out the right library to stick to and the right recepies to use. Unlike other domain where computational methods have seen a rapid growth, social sciences remain a relatively unexplored area. This is first of the 4 tutorials which will explore data analysis in education. 

There are four parts to soscipy:
- **Data Analysis** : Aims to make rapid analysis easy while not compromising on any functionalities and extendability
- **Data Processing** : Makes common actions with structured data easy and accessible without needing expertiese in computer science
- **Data Visualisation** : Rapid visualisations while ensuring that the output is publication quality
- **Utilities** : A set of utilities that you can plug and play to make your workflow easy

In [2]:
#Installation
!pip install --upgrade soscipy

Requirement already up-to-date: soscipy in /Users/saurabhkarn/opt/anaconda3/lib/python3.8/site-packages (0.0.21)


In [70]:
import pandas as pd
from soscipy.process import dfops

In [71]:
f1_path = 'https://github.com/karnsaurabhkumar/soscipy/blob/master/data/gyan_data.csv?raw=true'
f2_path = 'https://github.com/karnsaurabhkumar/soscipy/blob/master/data/gyan_data.csv?raw=true'

In [72]:
f1 = pd.read_csv(f1_path)
f2 = pd.read_csv(f2_path)

In [74]:
temp = dfops.combine(f1,f2)

In [77]:
dfops.string_matcher??